<a href="https://colab.research.google.com/github/cesar-yoab/Yield-Curves/blob/main/APM466.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from scipy.stats import binom

In [ ]:
import numpy as np

np.random.randint(0, 9999)

7458

In [ ]:
def even(max=100):
    val = 1
    while val < max:
        yield val*2
        val += 1

def odd(start=1, max=100000):
    val = start*2 - 1
    while val < max:
        yield val
        val = val + 2

def find_cum_sum_index(start, bound):
    #print("start", start)
    cum_sum = 0
    index = 0
    for n in odd(start):
        cum_sum += 1/n
        #print("sum, bound", cum_sum, bound)
        if cum_sum >= bound:
            return start + index
        index += 1

def find_pattern():
    pattern = []

    for n_i in even(6):
        bound = 1/n_i + 1
        if len(pattern) == 0:
            index = find_cum_sum_index(1, bound)
            #print(index)
        else:
            index = find_cum_sum_index(pattern[-1], bound)
        pattern.append(index)

    return pattern

In [ ]:
indexes = find_pattern()

[3, 25, 248, 2344, 21146]

In [ ]:
from datetime import datetime, timedelta

In [ ]:
(datetime.strptime('2022-02-01', "%Y-%m-%d") +
         timedelta(days=1)).strftime("%Y-%m-%d")

'2022-02-02'

In [ ]:
'user::passw:ord'.split('::')

['user', 'passw:ord']

In [ ]:
from itertools import combinations

sums = set()

c5 = combinations([2,5,8,11,14,17,20], 5)
c6 = combinations([2,5,8,11,14,17,20], 6)
c7 = [2,5,8,11,14,17,20]

for i in c5:
    sums.add(sum(i))

for i in c6:
    sums.add(sum(i))

sums.add(sum(c7))

sums

{40, 43, 46, 49, 52, 55, 57, 58, 60, 61, 63, 64, 66, 67, 69, 70, 72, 75, 77}

In [ ]:
import numpy as np

A = np.array([[2, 3, 2, 10, 0],[7, 3, 3, 2, -1],[2, 0, 1, 4, -1],[-1, 8, 5, 3, -1]])
B = np.array([[0, -3, -1, -7, 0], [5, 1, 2, 1, 2], [3, 0, 1, 5, 4], [0, 7, 6, 2, 5]])

A - B

array([[ 2,  6,  3, 17,  0],
       [ 2,  2,  1,  1, -3],
       [-1,  0,  0, -1, -5],
       [-1,  1, -1,  1, -6]])

In [ ]:
class A:
    def __init__(self, val):
        self.val = val

    def caller(self):
        getattr(A, "test_func")(self)

    def test_func(self):
        print(f"Val: {self.val}")


a = A(5)
a.caller()

Val: 5


In [ ]:
for i in range(10, 40):
    print(f"N - {i} probability: {binom.pmf(10, i, 0.5)}")

N - 10 probability: 0.0009765625
N - 11 probability: 0.00537109375
N - 12 probability: 0.016113281249999962
N - 13 probability: 0.034912109374999875
N - 14 probability: 0.06109619140624999
N - 15 probability: 0.0916442871093751
N - 16 probability: 0.12219238281250028
N - 17 probability: 0.14837646484375044
N - 18 probability: 0.16692352294921842
N - 19 probability: 0.17619705200195324
N - 20 probability: 0.17619705200195293
N - 21 probability: 0.16818809509277344
N - 22 probability: 0.15417242050170948
N - 23 probability: 0.13638329505920457
N - 24 probability: 0.11689996719360303
N - 25 probability: 0.09741663932800224
N - 26 probability: 0.0791510194540019
N - 27 probability: 0.06285522133111898
N - 28 probability: 0.04888739436864885
N - 29 probability: 0.037308800965547236
N - 30 probability: 0.027981600724160505
N - 31 probability: 0.0206530862487852
N - 32 probability: 0.01502042636275274
N - 33 probability: 0.010775523260235885
N - 34 probability: 0.007632662309333558
N - 35 pro

In [ ]:
def is_valid_sequence(degrees):
    s = sum(degrees)
    n = len(degrees)

    return s <= n*(n-1)

is_valid_sequence([7, 7, 7, 7, 7, 7, 6])

False

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from sympy.ntheory.factor_ import totient


# New Section

In [ ]:
sum([7, 5, 5, 5, 3, 3, 1, 1])

30

In [ ]:
8*7

56

In [ ]:
def isbn_calculate(nums):
    rsum = 0
    for i, n in enumerate(nums):
        if (i+1) % 2 == 0:
            rsum += 3*n
        else:
            rsum += n
    
    return rsum


def is_valid_isbn(nums):
    isbn = isbn_calculate(nums)
    return isbn % 10 == 0

def run_trial(ntrials=3):
    print("==== Starting Trials ====")
    for k in range(ntrials):
        nums = [9, 7, 8, 1, 4, 3, 9, 8, 8, 5, 2, 5, 3]
        #i, j = np.random.randint(0, 13, 2)
        i, j = 3, 6
        if i == j:
            i += 1
        print()
        print(f"== Trial {k} - Removing {i+1} and {j+1}")
        print(f"ORIGINAL ISBN:\t  {nums}")

        for n1 in range(0, 10):
            for n2 in range(0, 10):
                nums[i], nums[j] = n1, n2
                if is_valid_isbn(nums):
                    print(f"Found valid isbn: {nums} - {isbn_calculate(nums)}")


In [ ]:
isbn_calculate([9, 7, 8, 1, 4, 3, 9, 8, 8, 5, 2, 5, 3])

130

In [ ]:
a, b = np.random.randint(0, 13, 2)

In [ ]:
def euclidean_algorithm(x, y):
    import math

    while True:
        m = int(math.modf(x/y)[1])
        r = x % y
        print(f"{x} = {y} * {m} + {r}")
        if r == 1 or r == 0:
            break

        x = y
        y = r

In [ ]:
euclidean_algorithm(659880, 159853)

In [ ]:
def factor_integer(N, a=2):
    print("="*10)
    i = 2
    while True:
        a = (a**i) % N
        d = np.gcd(a-1, N)

        #print(f"a = {a}^{i} mod {N}   and   {d}=gcd(a-1, {N})")
        print(f"2^{i}! - 1 = {a-1} mod({N}),   gcd({a-1}, {N})={d}")

        if d > 1 and d < N:
            print("="*10)
            return f"Prime factor: {d}"
        
        i += 1

In [ ]:
print(factor_integer(12191))

2^2! - 1 = 3 mod(12191),   gcd(3, 12191)=1
2^3! - 1 = 63 mod(12191),   gcd(63, 12191)=1
2^4! - 1 = 2399 mod(12191),   gcd(2399, 12191)=1
2^5! - 1 = 3730 mod(12191),   gcd(3730, 12191)=1
2^6! - 1 = 6278 mod(12191),   gcd(6278, 12191)=73
Prime factor: 73


In [ ]:
1 ^ 0

1

In [ ]:
counts = {k:0 for k in range(4)}

for i in range(1, 61):
    counts[i % 4] += 1

counts

{0: 15, 1: 15, 2: 15, 3: 15}

In [ ]:
def factorial(x):
    if x == 1:
        return 1
    else:
        return x * factorial(x-1)

In [ ]:
count = 100

for i in range(1, 101):
    if i % 7 == 0:
        count -= 1
    elif i % 5 == 0:
        count -= 1

count

68

In [ ]:
def generate_data():
    data = {}

    # Generate timestamps
    tstamp = []
    months = {
        1: 31,
        2:28,
        3: 31,
        4:30,
        5:31,
        6:30
    }

    for m in range(1, 7):
        for d in range(1, months[m]+1):
            for h in range(10, 23):
                tstamp.append(dt.datetime(2021, m, d, h))

    subtotal = np.random.normal(450, 100, size=len(tstamp))
    imp = subtotal * 0.15
    total = subtotal + imp

    data['Timestamp'] = tstamp
    data['SubTotal'] = np.round(subtotal, 2)
    data['Imp'] = np.round(imp, 2)
    data['Total'] = np.round(total, 2)

    return pd.DataFrame(data)

In [ ]:
data = generate_data
da

In [ ]:
data.to_csv('fake_data.csv', index=False)

In [ ]:
bin(0)

'0b1010'

In [ ]:
10^0

10

In [ ]:
nums = [12, 16, 28]
for i in range(3):
    for j in range(3):
        nums = [12, 16, 28]
        nums[i] += 1
        nums[j] += 1
        nim = 0
        for n in nums:
            nim ^= n
        print(nums, "=> ", nim)

[14, 16, 28] =>  2
[13, 17, 28] =>  0
[13, 16, 29] =>  0
[13, 17, 28] =>  0
[12, 18, 28] =>  2
[12, 17, 29] =>  0
[13, 16, 29] =>  0
[12, 17, 29] =>  0
[12, 16, 30] =>  2


In [ ]:
12^16^28

0

In [ ]:
bin(ord('.'))

'0b101110'

In [ ]:
kpd = 1e10 * np.power(60, 2) * 24

In [ ]:
np.power(2, 55) / kpd

41.69999654972681

In [ ]:
np.power(2, 55) / 1e10

3602879.701896397

In [ ]:
kpy = kpd * 365.25
kpy

3.15576e+17

In [ ]:
kpy * 100

3.15576e+19

In [ ]:
np.log2(kpy*100)

64.77462129339004

In [ ]:
from yfinance import Ticker

In [ ]:
msft = Ticker("CDA 2023")

# get stock info
msft.info


{'logo_url': '', 'preMarketPrice': None, 'regularMarketPrice': None}